Data Collection

In [97]:
# Import packages and import data
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cmx
import matplotlib.colors as colors
import math
import pylab
import scipy.stats as stats
%matplotlib inline
url = 'https://raw.githubusercontent.com/q-x-yang/Datasets/master/listings-1.csv'
df = pd.read_csv(url)
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,...,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,2.019100e+13,10/14/19,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under const...,"Newly remodeled, modern, and bright garden uni...",New update: the house next door is under const...,none,*Quiet cul de sac in friendly neighborhood *St...,Due to the fact that we have children and a do...,*Public Transportation is 1/2 block away. *Ce...,*Full access to patio and backyard (shared wit...,A family of 4 lives upstairs with their dog. N...,* No Pets - even visiting guests for a short t...,NaN,NaN,https://a0.muscache.com/im/pictures/b7c2a199-4...,NaN,1169,https://www.airbnb.com/users/show/1169,Holly,7/31/08,"San Francisco, California, United States",We are a family with 2 boys born in 2009 and 2...,within a day,100%,NaN,t,https://a0.muscache.com/im/pictures/efdad96a-3...,https://a0.muscache.com/im/pictures/efdad96a-3...,Duboce Triangle,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"San Francisco, CA, United States",Duboce Triangle,Western Addition,...,$25.00,1,30,1,1,30,30,1.0,30.0,2 months ago,t,3,8,16,85,10/14/19,217,52,7/23/09,9/21/19,97.0,10.0,10.0,10.0,10.0,10.0,9.0,t,STR-0001256,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1,0,0,1.74
1,3850,https://www.airbnb.com/rooms/3850,2.019100e+13,10/14/19,Charming room for two,Your own private room plus access to a shared ...,This room can fit two people. Nobody else will...,Your own private room plus access to a shared ...,none,"This is a quiet, safe neighborhood on a substa...",House Rule footnotes: 1.\tI don’t allow check ...,Public transit service to my house is outstand...,"During the hours of 7:00 to 22:00, your access...","I live at the house in my own bedroom, and I w...",Airbnb has no default house rules that hosts c...,NaN,NaN,https://a0.muscache.com/im/pictures/b6e97d2a-1...,NaN,4921,https://www.airbnb.com/users/show/4921,Kevin,12/8/08,"San Francisco, California, United States",I am a retired software developer now raising ...,within an hour,100%,NaN,t,https://a0.muscache.com/im/pictures/97d29791-d...,https://a0.muscache.com/im/pictures/97d29791-d...,Inner Sunset,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"San Francisco, CA, United States",Inner Sunset,Inner Sunset,...,$20.00,1,5,1,1,5,5,1.0,5.0,3 days ago,t,5,32,62,62,10/14/19,160,36,7/16/09,9/30/19,94.0,10.0,10.0,10.0,10.0,10.0,10.0,t,STR-0000346,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,3,0,3,0,1.28
2,5858,https://www.airbnb.com/rooms/5858,2.019100e+13,10/14/19,Creative Sanctuary,NaN,We live in 

In [98]:
df.shape

(8111, 106)

Data Organization

In [99]:
for i in df.columns[:]:
  print(i)

id
listing_url
scrape_id
last_scraped
name
summary
space
description
experiences_offered
neighborhood_overview
notes
transit
access
interaction
house_rules
thumbnail_url
medium_url
picture_url
xl_picture_url
host_id
host_url
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_acceptance_rate
host_is_superhost
host_thumbnail_url
host_picture_url
host_neighbourhood
host_listings_count
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
street
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
city
state
zipcode
market
smart_location
country_code
country
latitude
longitude
is_location_exact
property_type
room_type
accommodates
bathrooms
bedrooms
beds
bed_type
amenities
square_feet
price
weekly_price
monthly_price
security_deposit
cleaning_fee
guests_included
extra_people
minimum_nights
maximum_nights
minimum_minimum_nights
maximum_minimum_nights
minimum_maximum_nights
maximum_maximum_nights
minimum_ni

In [100]:
df.isnull().sum()

id                                                 0
listing_url                                        0
scrape_id                                          0
last_scraped                                       0
name                                               0
                                                ... 
calculated_host_listings_count                     0
calculated_host_listings_count_entire_homes        0
calculated_host_listings_count_private_rooms       0
calculated_host_listings_count_shared_rooms        0
reviews_per_month                               1605
Length: 106, dtype: int64

In [101]:
def missing(df):
    print(round((df.isnull().sum() * 100/ len(df)),2).sort_values(ascending=False).head(30))
missing(df)

neighbourhood_group_cleansed    100.00
host_acceptance_rate            100.00
thumbnail_url                   100.00
xl_picture_url                  100.00
medium_url                      100.00
square_feet                      98.47
monthly_price                    87.13
weekly_price                     86.82
notes                            38.32
license                          37.12
access                           35.89
interaction                      32.35
transit                          29.77
house_rules                      26.83
host_about                       25.04
neighborhood_overview            24.88
security_deposit                 20.86
review_scores_value              20.40
review_scores_location           20.40
review_scores_checkin            20.40
review_scores_cleanliness        20.39
review_scores_accuracy           20.39
review_scores_communication      20.38
review_scores_rating             20.36
reviews_per_month                19.79
last_review              

In [102]:
for i in df.columns:
    print (i + ', Number of Missing Values:', len(df[i][df[i].isnull()]))

id, Number of Missing Values: 0
listing_url, Number of Missing Values: 0
scrape_id, Number of Missing Values: 0
last_scraped, Number of Missing Values: 0
name, Number of Missing Values: 0
summary, Number of Missing Values: 225
space, Number of Missing Values: 1134
description, Number of Missing Values: 50
experiences_offered, Number of Missing Values: 0
neighborhood_overview, Number of Missing Values: 2018
notes, Number of Missing Values: 3108
transit, Number of Missing Values: 2415
access, Number of Missing Values: 2911
interaction, Number of Missing Values: 2624
house_rules, Number of Missing Values: 2176
thumbnail_url, Number of Missing Values: 8111
medium_url, Number of Missing Values: 8111
picture_url, Number of Missing Values: 0
xl_picture_url, Number of Missing Values: 8111
host_id, Number of Missing Values: 0
host_url, Number of Missing Values: 0
host_name, Number of Missing Values: 8
host_since, Number of Missing Values: 8
host_location, Number of Missing Values: 13
host_about

In [103]:
Unrelated_cols = ['id', 'last_scraped', 'listing_url', 'scrape_id', 'last_review', 'name', 'experiences_offered', 
                 'picture_url', 'host_id', 'host_url', 'host_name', 'host_thumbnail_url', 'host_picture_url','host_neighbourhood', 
                 'host_listings_count', 'host_total_listings_count', 'host_verifications',
                 'street','city', 'state', 'market', 'smart_location', 'country', 'country_code', 'is_location_exact', 'minimum_minimum_nights', 
                 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 
                 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'first_review', 'calendar_updated', 'has_availability', 
                 'availability_30', 'availability_60', 'availability_90', 'availability_365','calendar_last_scraped','requires_license', 'jurisdiction_names', 
                 'is_business_travel_ready', 'calculated_host_listings_count_entire_homes', 'reviews_per_month',
                 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'require_guest_profile_picture', 
                  'require_guest_phone_verification', 'host_location', 'host_about']
#All columns with more than 30% of data missing are grouped into missing columns and will be dropped from the dataset.
missing_cols = ['neighbourhood_group_cleansed', 'host_acceptance_rate', 'thumbnail_url', 'xl_picture_url', 'medium_url', 
                'square_feet', 'monthly_price', 'weekly_price', 'notes', 'license', 'access', 'interaction']
#Text columns will not be dropped from the dataset, rather they will used to conduct the Sentiment analysis using NLP.
Text_cols = ['summary', 'space', 'description', 'neighborhood_overview', 'transit','house_rules', 'host_about']
df1 = df.drop(columns=Unrelated_cols).drop(columns=missing_cols)

Data Definition

In [104]:
# Print summaries for the table again
print ("Rows     : " ,df1.shape[0])
print ("Columns  : " ,df1.shape[1])
print ("\nFeatures : \n" ,df1.columns.tolist())
print ("\nMissing values :  ", df1.isnull().sum().values.sum())
print ("\nUnique values :  \n",df1.nunique())

Rows     :  8111
Columns  :  44

Features : 
 ['summary', 'space', 'description', 'neighborhood_overview', 'transit', 'house_rules', 'host_since', 'host_response_time', 'host_response_rate', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'zipcode', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price', 'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'cancellation_policy', 'calculated_host_listings_count']

Missing values :   24368

Unique values :  
 summary                           6523
space                             5697
descript

Data Definition

In [105]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8111 entries, 0 to 8110
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   summary                         7886 non-null   object 
 1   space                           6977 non-null   object 
 2   description                     8061 non-null   object 
 3   neighborhood_overview           6093 non-null   object 
 4   transit                         5696 non-null   object 
 5   house_rules                     5935 non-null   object 
 6   host_since                      8103 non-null   object 
 7   host_response_time              7184 non-null   object 
 8   host_response_rate              7184 non-null   object 
 9   host_is_superhost               8103 non-null   object 
 10  host_has_profile_pic            8103 non-null   object 
 11  host_identity_verified          8103 non-null   object 
 12  neighbourhood                   81

In [106]:
df1.describe()

,latitude,longitude,accommodates,bathrooms,bedrooms,beds,guests_included,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count
count,8111.000000,8111.000000,8111.000000,8099.000000,8107.000000,8102.000000,8111.000000,8.111000e+03,8.111000e+03,8111.000000,8111.000000,6460.000000,6457.000000,6457.000000,6456.000000,6458.000000,6456.000000,6456.000000,8111.000000
mean,37.766054,-122.430107,3.160769,1.395975,1.345874,1.744754,1.718407,1.234526e+04,1.284597e+04,45.203181,13.712736,95.421517,9.752207,9.603686,9.847584,9.817745,9.682776,9.375155,22.329799
std,0.022937,0.026967,1.896988,0.923213,0.925298,1.158799,1.334492,1.110357e+06,1.110352e+06,78.437987,22.523539,7.571031,0.732872,0.839768,0.593539,0.659274,0.715906,0.879174,54.369225
min,37.704630,-122.513060,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000e+00,1.000000e+00,0.000000,0.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000
25%,37.751450,-122.442830,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000e+00,2.900000e+01,1.000000,0.000000,95.000000,10.000000,9.000000,10.000000,10.000000,10.000000,9.000000,1.000000
50%,37.769150,-122.424650,2.000000,1.000000,1.000000,1.000000,1.000000,4.000000e+00,1.800000e+02,10.000000,2.000000,98.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,2.000000
75%,37.785670,-122.410615,4.000000,1.500000,2.000000,2.000000,2.000000,3.000000e+01,1.125000e+03,54.000000,18.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
max,37.828790,-122.368570,16.000000,14.000000,14.000000,14.000000,16.000000,1.000000e+08,1.000000e+08,757.000000,258.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,248.000000


Data Cleaning

In [107]:
# host_response_rate has to be changed to int from object
df1['host_response_rate'] = df1['host_response_rate'].str.rstrip('%').astype('float') / 100.0

In [108]:
# zipcode has be to be changed from object to int64
df1['zipcode'].unique()

array(['94117', '94131', '94110', '94107', '94109', '94102', '94114',
       '94105', '94133', '94115', '94121', '94118', '94127', '94104',
       '94122', '94123', '94112', '94108', '94111', '94103', '94124',
       '94113', '94116', '94129', '94132', nan, '94134', '94158', '94014',
       'CA 94108', 'CA 94107', 'CA 94158', 'CA 94103', 'CA 94115',
       'CA 94109', 'CA 94117', 'CA 94102', 'CA 94131', 'CA 94105', 'CA',
       'CA 94110', 'CA 94123', 'CA 94114', '94015', '94130'], dtype=object)

In [109]:
df1['zipcode_new'] = df['zipcode'].str.extract('(\d+)').astype(float)
df1['zipcode_new'].unique()

array([94117., 94131., 94110., 94107., 94109., 94102., 94114., 94105.,
       94133., 94115., 94121., 94118., 94127., 94104., 94122., 94123.,
       94112., 94108., 94111., 94103., 94124., 94113., 94116., 94129.,
       94132.,    nan, 94134., 94158., 94014., 94015., 94130.])

In [110]:
drop_cols = ['zipcode']

In [111]:
# price columns which include price security_deposit, cleaning_fee, extra people all have "$" sign in front of all the numbers. We will need to remove the dollar sign
# and then remove anything that isn't a number
import re
price_cols = ['price','cleaning_fee','security_deposit','extra_people']
for col in price_cols:
    df1[col] = df1[col].fillna("0")
    df1[col] = df1[col].apply(lambda x: float(re.compile('[^0-9eE.]').sub('', x)) if len(x)>0 else 0)

In [112]:
# host_since column has to be converted to date time format
df1['host_since'] = pd.to_datetime(df1['host_since'])

In [113]:
#Re-run the data types for the dataset and scrutinize if there's any other column that has inappropriate data type
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8111 entries, 0 to 8110
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   summary                         7886 non-null   object        
 1   space                           6977 non-null   object        
 2   description                     8061 non-null   object        
 3   neighborhood_overview           6093 non-null   object        
 4   transit                         5696 non-null   object        
 5   house_rules                     5935 non-null   object        
 6   host_since                      8103 non-null   datetime64[ns]
 7   host_response_time              7184 non-null   object        
 8   host_response_rate              7184 non-null   float64       
 9   host_is_superhost               8103 non-null   object        
 10  host_has_profile_pic            8103 non-null   object        
 11  host

In [114]:
df1.sample(10)

,summary,space,description,neighborhood_overview,transit,house_rules,host_since,host_response_time,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,calculated_host_listings_count,zipcode_new
4656,Beautiful and fully furnished living space in ...,"Perfect for couples, families and business peo...",Beautiful and fully furnished living space in ...,NaN,NaN,Please consider the neighbors--no parties or l...,2014-07-30,within an hour,1.00,t,t,f,Mission Terrace,Outer Mission,94112,37.73034,-122.43148,Guest suite,Entire home/apt,5,1.0,2.0,3.0,Real Bed,"{Wifi,""Free street parking"",Heating,""Family/ki...",179.0,0.0,80.0,3,30.0,1,90,65,61,96.0,10.0,10.0,10.0,10.0,10.0,9.0,f,moderate,1,94112.0
1593,"Amazing 2 bedroom, 2 bathroom corporate apartm...",2 bedrooms each with their own queen bed and f...,"Amazing 2 bedroom, 2 bathroom corporate apartm...","The best part is location, location, location....",-Only 1 to 5 blocks at most from the train sta...,Have a great time!,2014-05-26,within an hour,1.00,t,t,t,South Beach,South of Market,94105,37.78699,-122.39130,Apartment,Entire home/apt,6,2.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Wifi,Pool,Kitchen,""Paid parking...",173.0,5000.0,160.0,4,25.0,30,365,12,6,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,moderate,1,94105.0
7332,"You’ll love my place because of the location, ...",Our hotel consists a mix of rooms with Private...,"You’ll love my place because of the location, ...","Just a heads up, we are right in the middle of...",All types of public transportation right outsi...,- No Visitors in the building after 9PM. - No...,2016-11-15,within an hour,1.00,f,f,f,SoMa,South of Market,94103,37.77157,-122.41080,Boutique hotel,Hotel room,2,4.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Buzzer/wireless ...",119.0,250.0,15.0,2,50.0,7,1125,8,8,93.0,9.0,9.0,9.0,9.0,8.0,10.0,t,strict_14_with_grace_period,10,94103.0
143,This is the ground floor in-law unit that can ...,The perfect place for a single person or coupl...,This is the ground floor in-law unit that can ...,If you are interested in exploring the neighbo...,"Regarding public transportation, there are 3 b...",Basic Rules: Check out time is 11 am. You wil...,2011-06-16,within an hour,1.00,f,t,t,Twin Peaks,Twin Peaks,94131,37.75195,-122.44928,Apartment,Entire home/apt,2,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Free par...",115.0,200.0,75.0,2,15.0,30,30,90,41,82.0,8.0,8.0,9.0,9.0,9.0,8.0,f,strict_14_with_grace_period,3,94131.0
7855,"A beautiful, private, 1-bedroom suite in a saf...",Whether you will be here seeing relatives livi...,"A beautiful, private, 1-bedroom suite in a saf...",The studio is located in one of San Francisco'...,"~ easy access to highways I-280, US-101, and ...",Quiet hours 10pm to 9am (9pm to 9am outside ar...,2018-04-16,within a few hours,1.00,f,t,f,Sunnyside,Outer Mission,94127,37.73379,-122.45201,Guest suite,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Wifi,""Free street parking"",Heat...",130.0,0.0,40.0,1,0.0,2,10,3,3,100.0,9.0,10.0,10.0,10.0,10.0,9.0,f,moderate,1,94127.0
6669,"Featured in The New York Times, The Wall Stree...",- In-suite laundry - Fully-equipped kitchen - ...,"Featured in The New York Times, The Wall Stree...",Rincon Hill highlights - Walkability score - ...,NaN,House Rules 1. All bookings require a security...,2018-10-10,within an hour,0.93,f,t,f,South Beach,South of Market,94105,37.78733,-122.39443,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitch

In [115]:
#Let's look at the residual columns with missing values
#First look at numeric columns that have missing values
for y in df1.columns:
    if(df1[y].dtype == np.float64 or df1[y].dtype == np.int64):
      if(df1[y].isna().sum()>0):
          print(y, df1[y].isna().sum())

host_response_rate 927
bathrooms 12
bedrooms 4
beds 9
review_scores_rating 1651
review_scores_accuracy 1654
review_scores_cleanliness 1654
review_scores_checkin 1655
review_scores_communication 1653
review_scores_location 1655
review_scores_value 1655
zipcode_new 246


In [116]:
#For host_response_rate, bathrooms, bedrooms, beds, fill in the NA with 0
df1['host_response_rate'] = df1['host_response_rate'].fillna(0)
df1['bathrooms'] = df1['bathrooms'].fillna(0) 
df1['bedrooms'] = df1['bedrooms'].fillna(0) 
df1['beds'] = df1['beds'].fillna(0)

In [117]:
#For the rest of the columns, drop the rows with NA
df1 = df1.dropna(how='any', subset=['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
                                    'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                                    'review_scores_value', 'zipcode_new'])

In [118]:
#Then we can proceed to check descriptive columns that have missing values
for y in df1.columns:
    if(df1[y].dtype == np.object):
      if(df1[y].isna().sum()>0):
          print(y, df1[y].isna().sum())

summary 175
space 789
description 18
neighborhood_overview 1301
transit 1367
house_rules 1409
host_response_time 678
host_is_superhost 8
host_has_profile_pic 8
host_identity_verified 8


In [121]:
# Fill out the missing descriptions with "N/A", for host_response_time, fill out the missing descriptions as 'a few days or more',
# For Y/N questions like 'host_is_superhost', fill out the missing values as 'f'
df1['summary'] = df1['summary'].fillna('N/A')
df1['space'] = df1['space'].fillna('N/A')
df1['description'] = df1['description'].fillna('N/A')
df1['neighborhood_overview'] = df1['neighborhood_overview'].fillna('N/A')
df1['transit'] = df1['transit'].fillna('N/A')
df1['house_rules'] = df1['house_rules'].fillna('N/A')
df1['host_response_time'] = df1['host_response_time'].fillna('a few days or more')
df1['host_is_superhost'] = df1['host_is_superhost'].fillna('f')
df1['host_has_profile_pic'] = df1['host_has_profile_pic'].fillna('f')
df1['host_identity_verified'] = df1['host_identity_verified'].fillna('f')

In [125]:
#For datetime42 type colume "host_since", drop the missing values
df1 = df1.dropna(how='any', subset=['host_since'])

In [126]:
#Check for Any remaining missing value
df1.isna().sum()[df1.isna().sum()>0]

Series([], dtype: int64)